In [54]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [55]:
import os
import sys
import random
import numpy as np
import pandas as pd
import pickle as pkl
import matplotlib.pyplot as plt
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

In [56]:
torch.cuda.is_available(), torch.cuda.device_count()

(True, 1)

In [57]:
ic50_data_path = "GDSC2_fitted_dose_response_27Oct23.xlsx"
raw_data_path = "GDSC2_public_raw_data_27Oct23.csv"

In [58]:
ic50_data = pd.read_excel(ic50_data_path)

In [59]:
raw_data = pd.read_csv(raw_data_path)

In [60]:
cell_line_names_to_cosmic_ids = {}
cosmic_ids_to_cell_line_names = {}

for cosmic_id, cell_line_name in zip(ic50_data["COSMIC_ID"].tolist(), ic50_data["CELL_LINE_NAME"].tolist()):
    cell_line_names_to_cosmic_ids[cell_line_name] = cosmic_id
    cosmic_ids_to_cell_line_names[cosmic_id] = cell_line_name

cell_line_names = list(cell_line_names_to_cosmic_ids.keys())
cosmic_ids = list(cosmic_ids_to_cell_line_names.keys())

for cosmic_id, cell_line_name in zip(ic50_data["COSMIC_ID"].tolist(), ic50_data["CELL_LINE_NAME"].tolist()):
    assert cell_line_names_to_cosmic_ids[cell_line_name] == cosmic_id
    assert cosmic_ids_to_cell_line_names[cosmic_id] == cell_line_name

In [61]:
drug_names_to_drug_ids = {}
drug_ids_to_drug_names = {}

for drug_id, drug_name in zip(ic50_data["DRUG_ID"].tolist(), ic50_data["DRUG_NAME"].tolist()):
    drug_ids_to_drug_names[drug_id] = drug_name

drug_names = set(list(drug_ids_to_drug_names.values()))

for drug_name in drug_names:
    drug_names_to_drug_ids[drug_name] = []

for drug_id, drug_name in zip(ic50_data["DRUG_ID"].tolist(), ic50_data["DRUG_NAME"].tolist()):
    if not (drug_id in drug_names_to_drug_ids[drug_name]):
        drug_names_to_drug_ids[drug_name].append(drug_id)

drug_ids = list(drug_ids_to_drug_names.keys())

for drug_id, drug_name in zip(ic50_data["DRUG_ID"].tolist(), ic50_data["DRUG_NAME"].tolist()):
    assert drug_id in drug_names_to_drug_ids[drug_name]
    assert drug_name == drug_ids_to_drug_names[drug_id]

In [62]:
n_cell_lines = len(cell_line_names)
n_drugs = len(drug_ids)

In [63]:
n_cell_lines, n_drugs

(969, 295)

In [118]:
times_sampled = np.zeros((n_cell_lines, n_drugs))

for cell_line_name, drug_id in zip(ic50_data["CELL_LINE_NAME"].tolist(), ic50_data["DRUG_ID"].tolist()):
    times_sampled[cell_line_names.index(cell_line_name), drug_ids.index(drug_id)] += 1

ln_ic50s = np.zeros((n_cell_lines, n_drugs))

for cell_line_name, drug_id, ln_ic50 in zip(ic50_data["CELL_LINE_NAME"].tolist(), ic50_data["DRUG_ID"].tolist(), ic50_data["LN_IC50"]):
    ln_ic50s[cell_line_names.index(cell_line_name), drug_ids.index(drug_id)] = ln_ic50

ln_ic50s[times_sampled == 0] = np.nan
for cell_line_name in cell_line_names:
    for drug_id in drug_ids:
        if np.isnan(ln_ic50s[cell_line_names.index(cell_line_name), drug_ids.index(drug_id)]):
            ln_ic50s[cell_line_names.index(cell_line_name), drug_ids.index(drug_id)] = np.nanmean(ln_ic50s[:,drug_ids.index(drug_id)])

In [120]:
raw_data.DRUG_ID = raw_data.DRUG_ID.fillna(-1.0).astype(int)

n_drugs_raw = len(set(raw_data.DRUG_ID.tolist())) - 1

drug_ids_raw = list(set(raw_data.DRUG_ID.tolist()))
cosmic_ids_raw = list(set(raw_data.COSMIC_ID.tolist()))

for raw_id in cosmic_ids_raw:
  assert raw_id in cosmic_ids
for id in cosmic_ids:
  assert id in cosmic_ids_raw

missing_drug_ids = []

for raw_id in drug_ids_raw:
  if raw_id > 0 and not (raw_id in drug_ids):
    missing_drug_ids.append(raw_id)

for id in drug_ids:
  assert id in drug_ids_raw

In [121]:
barcodes = list(set(raw_data["BARCODE"].values))

n_barcodes = len(barcodes)

In [122]:
n_barcodes

16354

In [123]:
drug_ids_raw.sort()
cosmic_ids.sort()

In [124]:
def retrieve_dose_intensity_data(cosmic_id, drug_id):
    assert cosmic_id in cosmic_ids, "Invalid COSMIC ID, please try again."
    assert drug_id in drug_ids_raw, "Drug ID not present in raw data, please try again."
    subset = raw_data[(raw_data["COSMIC_ID"] == demo_cosmic_id) & (raw_data["DRUG_ID"] == demo_drug_id)]
    subset_barcodes = set(subset["BARCODE"].values)
    data = []
    for barcode in subset_barcodes:
        further_subset = subset[subset["BARCODE"] == barcode]
        tags = further_subset["TAG"].values
        tags_passed = True
        dose_counts = [0] * 7
        for i, tag in enumerate(tags):
            if i % 7 == 0:
                for count in dose_counts: assert count == i // 7, "Doses not contiguous."
            split_tag = tag.split("-")
            if split_tag[0][0] != 'L' or split_tag[-1] != "S":
                tags_passed = False
                break
            dose_counts[int(split_tag[1][1]) - 1] += 1
        if not tags_passed: continue
        for count in dose_counts:
            assert count == dose_counts[0], f"Doses unevenly sampled for barcode {barcode}!"
        for i in range(dose_counts[0]):
            plate_data = {"barcode": barcode,
                         "seeding_density": further_subset["SEEDING_DENSITY"].iloc[0],
                         "dose": further_subset["CONC"].values[7*i:7*(i+1)],
                         "intensity": further_subset["INTENSITY"].values[7*i:7*(i+1)],
                         "position": further_subset["POSITION"].values[7*i:7*(i+1)]}
            data.append(plate_data)
    return data

In [125]:
def retrieve_baseline_data(barcode):
    assert barcode in barcodes, "Invalid barcode. Please try again."
    subset = raw_data[(raw_data["BARCODE"] == barcode) & (raw_data["TAG"] == "B")]
    plate_data = {"barcode": barcode,
                 "seeding_density": subset["SEEDING_DENSITY"].iloc[0],
                 "intensity": subset["INTENSITY"].values,
                 "position": subset["POSITION"].values}
    return plate_data

In [126]:
def retrieve_untreated_data(barcode):
    assert barcode in barcodes, "Invalid barcode. Please try again."
    subset_0 = raw_data[(raw_data["BARCODE"] == barcode) & (raw_data["TAG"] == "NC-0")]
    subset_1 = raw_data[(raw_data["BARCODE"] == barcode) & (raw_data["TAG"] == "NC-1")]
    assert len(subset_0.index) > 0 and len(subset_1.index) > 0, "Missing data for either NC-0 or NC-1!"
    plate_data = {"barcode": barcode,
                 "seeding_density": subset_0["SEEDING_DENSITY"].iloc[0],
                 "intensity_0": subset_0["INTENSITY"].values,
                 "intensity_1": subset_1["INTENSITY"].values,
                 "position_0": subset_0["POSITION"].values,
                 "position_1": subset_1["POSITION"].values}
    return plate_data

In [127]:
neg_ctrl_data = np.load("neg-ctrl-priors.npz")

In [128]:
pos_ctrl_data = np.load("pos-ctrl-priors.npz")

In [129]:
neg_ctrl_data, pos_ctrl_data

(NpzFile 'neg-ctrl-priors.npz' with keys: barcodes, c,
 NpzFile 'pos-ctrl-priors.npz' with keys: barcodes, a, b, scores)

In [130]:
genomic_feature_matrix = pd.read_csv("combined_genetic_features_925.csv", index_col=0)

In [131]:
genomic_feature_matrix

,TP53,NRAS,NOTCH1,TET2,FLT3,PTPN11,NF1,FBXW7,KRAS,PTEN,...,chr8:22422534-22423702,chr6:6007387-6007797,chr5:180017099-180019062,chr11:65408344-65408631,chr16:68676450-68676843,chr17:42092143-42092432,chr20:30192854-30194025,chr3:50604905-50605359,chr4:102711829-102712199,chr5:125930601-125931049
1240121,1,-1,-1,-1,-1,1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1240122,1,-1,-1,-1,-1,-1,-1,1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1240123,1,-1,-1,-1,-1,-1,-1,1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1240125,1,-1,-1,-1,-1,-1,1,-1,-1,-1,...,0,0,0,0,0,0,0,0,0,0
1240127,1,-1,-1,-1,-1,-1,1,1,-1,1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
949179,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
971773,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,0,0,0,0,0,0,0,0,0,0
971777,-1,1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
998184,1,-1,-1,-1,-1,-1,-1,1,-1,1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [132]:
cosmic_ids_for_analysis = genomic_feature_matrix.index.values
cosmic_ids_for_analysis = np.sort(cosmic_ids_for_analysis)

In [133]:
for id in cosmic_ids_for_analysis:
    assert id in cosmic_ids

In [134]:
genomic_feature_names = genomic_feature_matrix.columns.values.tolist()

cg_feature_names = genomic_feature_names[:300]
cnv_feature_names = genomic_feature_names[300:300+1227]
meth_feature_names = genomic_feature_names[300+1227:]

n_features = len(genomic_feature_names)
n_cg_features = len(cg_feature_names)
n_cnv_features = len(cnv_feature_names)
n_meth_features = len(meth_feature_names)

In [135]:
n_cg_features, n_cnv_features, n_meth_features

(300, 1227, 378)

In [136]:
X = torch.Tensor(genomic_feature_matrix.values)

In [137]:
X.size()

torch.Size([925, 1905])

In [138]:
y = torch.Tensor(np.array([ln_ic50s[cell_line_names.index(cosmic_ids_to_cell_line_names[cosmic_id]),:] for cosmic_id in cosmic_ids_for_analysis]))

In [139]:
y.size()

torch.Size([925, 295])

In [140]:
y_flat = y.flatten()

In [141]:
y_flat[:295] == y[0]

tensor([True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, Tr

In [142]:
y.size()
# y[i] corr to drug i%295, cell line i//295

torch.Size([925, 295])

In [143]:
indices = np.arange(925*295)

In [144]:
train_indices, test_indices = train_test_split(indices, test_size=0.20, random_state=42, shuffle=True)

In [145]:
len(train_indices) == 925*295*4//5

True

In [146]:
train_indices

array([  4537,  44529, 204756, ..., 131932, 146867, 121958])

In [147]:
class IC50PredictionDataset(Dataset):

    def __init__(self, X, y, train_indices, test_indices, split="train"):
        self.X = X
        self.y = y
        self.train_indices = train_indices
        self.test_indices = test_indices
        self.split = split

    def __len__(self):
        return 925*295//5 if self.split == "test" else 925*295*4//5

    def __getitem__(self, idx):
        if self.split == "train":
            return X[self.train_indices[idx]//295], self.train_indices[idx]%295, self.y[self.train_indices[idx]]
        return X[self.test_indices[idx]//295], self.train_indices[idx]%295, self.y[self.test_indices[idx]]
        

In [148]:
train_dataset = IC50PredictionDataset(X, y_flat, train_indices, test_indices, split="train")
test_dataset = IC50PredictionDataset(X, y_flat, train_indices, test_indices, split="test")

In [149]:
len(train_dataset), len(test_dataset)

(218300, 54575)

In [150]:
batch_size = 2500
val_batch_size = 925

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
test_dataloader = DataLoader(test_dataset, batch_size=val_batch_size, shuffle=False, num_workers=2)

In [151]:
class NeuralIC50Predictor(nn.Module):

    def __init__(self, layers, dropout=0., batch_norm=True, input_dim=n_features, output_dim=1, embed_dim=128):
        super().__init__()
        all_layers = []
        prev_out = input_dim + embed_dim
        self.drug_embeddings = nn.Parameter(torch.zeros(n_drugs, embed_dim))
        for layer_size in layers:
            if dropout > 0.:
                all_layers.append(nn.Dropout(dropout))
            if batch_norm:
                all_layers.append(nn.BatchNorm1d(prev_out))
            all_layers.append(nn.Linear(prev_out, layer_size))
            all_layers.append(nn.ReLU())
            prev_out = layer_size
        all_layers.append(nn.Linear(prev_out, output_dim))
        self.mlp = nn.Sequential(*all_layers)

    def forward(self, x, d):
        return self.mlp(torch.cat((x, self.drug_embeddings[d]), dim=1))
        

In [152]:
predictor = NeuralIC50Predictor([100, 100], dropout=0.5, batch_norm=True, embed_dim=128)

In [153]:
for param in predictor.parameters():
    print(param.data.size())

torch.Size([295, 128])
torch.Size([2033])
torch.Size([2033])
torch.Size([1000, 2033])
torch.Size([1000])
torch.Size([1000])
torch.Size([1000])
torch.Size([200, 1000])
torch.Size([200])
torch.Size([200])
torch.Size([200])
torch.Size([200, 200])
torch.Size([200])
torch.Size([1, 200])
torch.Size([1])


In [154]:
lr=1e-3
wd=1e-3

optimizer = torch.optim.Adam(predictor.parameters(), lr=lr, weight_decay=wd)

In [155]:
loss_fn = torch.nn.MSELoss()

In [156]:
epochs = 100
steps_per_epoch = len(train_dataloader)
steps_per_test_epoch = len(test_dataloader)
log_freq = 40

losses = []
test_losses = []

predictor.cuda()

for n in range(epochs):

    predictor.train()

    running_loss = 0.
    log_loss = 0.
    
    print(f"----STARTING EPOCH {n+1}----")
    for i, (X_, d_, y_) in enumerate(train_dataloader):

        X_ = X_.cuda()
        d_ = d_.cuda()
        y_ = y_.cuda()
        
        optimizer.zero_grad()
        y_hat = predictor(X_, d_)
        loss = loss_fn(y_hat, y_.unsqueeze(-1))
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        log_loss += loss.item()

        if (i+1)%log_freq == 0:
            print(f"Epoch {n+1}, step {i+1}, loss {log_loss/log_freq:.4f}.")
            log_loss = 0.

    print(f"Epoch {n+1}, loss {running_loss/steps_per_epoch:.4f}.")
    losses.append(running_loss/steps_per_epoch)
    running_loss = 0.

    predictor.eval()

    running_loss = 0.
    
    for i, (X_, d_, y_) in enumerate(test_dataloader):

        X_ = X_.cuda()
        d_ = d_.cuda()
        y_ = y_.cuda()
        
        y_hat = predictor(X_, d_)
        loss = loss_fn(y_hat, y_.unsqueeze(-1))

        running_loss += loss.item()

    print(f"Epoch {n+1}, val loss {running_loss/steps_per_test_epoch:.4f}.")
    test_losses.append(running_loss/steps_per_test_epoch)
    running_loss = 0.
        
    

----STARTING EPOCH 1----


Exception ignored in: 
<function _MultiProcessingDataLoaderIter.__del__ at 0x2b2dde9ba790>Traceback (most recent call last):
  File "/home/spa2139/.conda/envs/torch1_gpu_exp/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1510, in __del__
    self._shutdown_workers()
  File "/home/spa2139/.conda/envs/torch1_gpu_exp/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1493, in _shutdown_workers
    if w.is_alive():
  File "/home/spa2139/.conda/envs/torch1_gpu_exp/lib/python3.9/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x2b2dde9ba790>
Traceback (most recent call last):
  File "/home/spa2139/.conda/envs/torch1_gpu_exp/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1510, in __del__
Exception ignored in: Exception ignored in: <function _Mul

Epoch 1, step 40, loss 4.9039.
Epoch 1, step 80, loss 2.0079.
Epoch 1, loss 3.3167.
Epoch 1, val loss 10.9522.
----STARTING EPOCH 2----


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x2b2dde9ba790>Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x2b2dde9ba790>
Exception ignored in: 
Traceback (most recent call last):
  File "/home/spa2139/.conda/envs/torch1_gpu_exp/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1510, in __del__
    self._shutdown_workers()<function _MultiProcessingDataLoaderIter.__del__ at 0x2b2dde9ba790>

Traceback (most recent call last):
  File "/home/spa2139/.conda/envs/torch1_gpu_exp/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1510, in __del__
      File "/home/spa2139/.conda/envs/torch1_gpu_exp/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1493, in _shutdown_workers
self._shutdown_workers()self._shutdown_workers()Traceback (most recent call last):
    
if w.is_alive():Exception ignored in:   File "/home/spa2139/.conda/envs/torch1_gpu_exp/lib/python3.9/site-packages/torch/utils/data/d

Epoch 2, step 40, loss 1.9530.
Epoch 2, step 80, loss 1.9360.
Epoch 2, loss 1.9389.
Epoch 2, val loss 11.9220.
----STARTING EPOCH 3----


...

Exception ignored in: Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x2b2dde9ba790><function _MultiProcessingDataLoaderIter.__del__ at 0x2b2dde9ba790>

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x2b2dde9ba790>Traceback (most recent call last):
Traceback (most recent call last):
Exception ignored in: 
  File "/home/spa2139/.conda/envs/torch1_gpu_exp/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1510, in __del__
<function _MultiProcessingDataLoaderIter.__del__ at 0x2b2dde9ba790>Traceback (most recent call last):
  File "/home/spa2139/.conda/envs/torch1_gpu_exp/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1510, in __del__

  File "/home/spa2139/.conda/envs/torch1_gpu_exp/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1510, in __del__
Traceback (most recent call last):
        self._shutdown_workers()    self._shutdown_workers()  File "/home/spa2139/.conda/envs/torch1_gpu

Epoch 10, step 40, loss 1.7559.
Epoch 10, step 80, loss 1.7548.
Epoch 10, loss 1.7606.
Epoch 10, val loss 11.8871.
----STARTING EPOCH 11----


Exception ignored in: Exception ignored in: Exception ignored in: Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x2b2dde9ba790><function _MultiProcessingDataLoaderIter.__del__ at 0x2b2dde9ba790><function _MultiProcessingDataLoaderIter.__del__ at 0x2b2dde9ba790><function _MultiProcessingDataLoaderIter.__del__ at 0x2b2dde9ba790>



Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/spa2139/.conda/envs/torch1_gpu_exp/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1510, in __del__
  File "/home/spa2139/.conda/envs/torch1_gpu_exp/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1510, in __del__
  File "/home/spa2139/.conda/envs/torch1_gpu_exp/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1510, in __del__
Traceback (most recent call last):
        self._shutdown_workers()      File "/home/spa2139/.conda/envs/torch1_gpu_exp/lib/python3.9/site-

Epoch 11, step 40, loss 1.7352.
Epoch 11, step 80, loss 1.7284.
Epoch 11, loss 1.7303.
Epoch 11, val loss 11.6784.
----STARTING EPOCH 12----


KeyboardInterrupt: 